In [1]:
import os

os.sys.path.append(os.path.join(os.getcwd(), "../.."))
from models.Batch import Batch
from databases.connector import Connector
from models.ExportFormat import ExportFormat
from prompting.get_themes_list import (
    selected_level_1_themes_first_version,
    get_accepted_themes_list_from_themes_list,
)
from metrics.batch import compute_batch_theme_counts
from utils.database import add_question_ids_to_batch

In [2]:
connector = Connector(ExportFormat.JSON)

In [3]:
SIZE = 1500
LEGISLATURE = 8
NUMBER_OF_BATCH_TO_CREATE = 1
THEMES_LIST, THEME_HIERARCHY_LEVEL = selected_level_1_themes_first_version()

In [ ]:
themes = connector.client.get_themes(
    {"level": THEME_HIERARCHY_LEVEL, "name": {"$in": THEMES_LIST}}
)

themes_list = list(themes)

accepted_themes_for_questions = get_accepted_themes_list_from_themes_list(themes_list)

questions = connector.client.get_random_questions(
    number_of_questions=SIZE,
    legislature=LEGISLATURE,
    accepted_themes=accepted_themes_for_questions,
)

question_ids = [x["id"] for x in questions]

comment = "selected level 1 themes first version 8th legislature only"

for i in range(0, NUMBER_OF_BATCH_TO_CREATE):
    batch = Batch(question_ids=question_ids, size=len(question_ids), comment=comment)

    inserted_batch = connector.client.add_batch(batch)

    print(f"Inserted Batch: {inserted_batch}")

In [ ]:
"""
USE THIS CELL TO EDIT THE BATCH
"""
# themes = connector.client.get_themes(
#     {"level": 1, "name": {"$in": ["numérique"]}}
# )

# themes_list = list(themes)

# accepted_themes_for_questions = get_accepted_themes_list_from_themes_list(themes_list)

# questions = connector.client.get_random_questions(
#     number_of_questions=250,
#     accepted_themes=accepted_themes_for_questions,
# )

# question_ids = [x["id"] for x in questions]

# add_question_ids_to_batch(question_ids, inserted_batch.inserted_id)

In [ ]:
compute_batch_theme_counts(inserted_batch.inserted_id, THEME_HIERARCHY_LEVEL)